In [ ]:
from sam2_class import SAM2Wrapper

checkpoint_path = '/home/admina/segmetation/DLOSeg/src/sam2_rt/checkpoints/sam2.1_hiera_small.pt'
model_cfg = '/home/admina/segmetation/DLOSeg/src/sam2_rt/sam2/configs/sam2.1/sam2.1_hiera_s.yaml'

In [5]:
sam2 = SAM2Wrapper(
    checkpoint_path=checkpoint_path,
)

NameError: name 'checkpoint_path' is not defined